# Cleaning and merging data for race datasets from 2013-2022 except 2018

- Combine data from 2013-2022.
- Isolate data for District 7.
- Calculate total population per census tract.
- Determine racial percentages within each tract.
- Add a 'Year' column to the dataset.
- Assign unique identifiers to duplicate names within each year.

In [5]:
import pandas as pd
import os
from collections import defaultdict

# All years except 2018
years = [year for year in range(2013, 2023) if year != 2018]

# Initialize an empty list to hold the results
all_results = []

def add_duplicate_numbering(labels,year):
    label_counts = defaultdict(int)
    modified_labels = []
    for label in labels:
        base_label = label.strip()
        label_counts[base_label] += 1
        if label_counts[base_label] > 1:
            modified_labels.append(f"{base_label} {label_counts[base_label]}")
        else:
            modified_labels.append(base_label)
    return modified_labels

for year in years:
    file_name = f"{year}.csv"
    file_path = os.path.join(file_name)
    
    print(f"Reading data from {file_path}...")
    try:
        # preserve the original order
        df = pd.read_csv(file_path, encoding='utf-8')
        
    except Exception as e:
        print(f"Error reading file {file_name}: {str(e)}")
        continue

    # adding year as column
    df['year'] = year
    
    # Getting all columns with 'Estimate' name in it 
    estimate_cols = [col for col in df.columns if 'Estimate' in col]

    results = []
    
    # Get total population for % calculation
    total_pop_row = df[df['Label (Grouping)'].str.strip() == "Total population"].iloc[0]
    total_pop = sum(
        float(str(total_pop_row[col]).replace(',', ''))
        for col in estimate_cols
        if pd.notna(total_pop_row[col])
    )
    
    # Process each row in original order
    for idx, row in df.iterrows():
        label = row['Label (Grouping)']
        
        # adding all estimates from each census tract as total
        total_estimate = 0
        for col in estimate_cols:
            try:
                value = str(row[col]).replace(',', '')
                if pd.notna(value):
                    total_estimate += float(value)
            except (ValueError, TypeError):
                continue
        
        # Calculate percentage
        if total_pop > 0 and total_estimate > 0:
            percentage = (total_estimate / total_pop * 100)
        else:
            percentage = 0
            
        # Append results
        results.append({
            'year': year,
            'label': label,
            'total_estimate': round(total_estimate, 1),
            'percentage': round(percentage, 1)
        })
    
    # result dataframe
    result_df = pd.DataFrame(results)
    all_results.append(result_df)

# Combined_df for all results
combined_df = pd.concat(all_results, ignore_index=True)

# Add numbering to duplicate labels for each year
combined_df['label'] = combined_df.groupby('year')['label'].transform(lambda x: add_duplicate_numbering(x, combined_df['year'].unique()))


# Preserve the original order of labels within each year
combined_df = combined_df.sort_values(['year', 'label'], key=lambda x: combined_df.index)

# Save csv file for the filtered/combined dataset to the folder
output_file = 'census_analysis_results.csv'
combined_df.to_csv(output_file, index=False)

combined_df.head(15)

Reading data from 2013.csv...
Reading data from 2014.csv...
Reading data from 2015.csv...
Reading data from 2016.csv...
Reading data from 2017.csv...
Reading data from 2019.csv...
Reading data from 2020.csv...
Reading data from 2021.csv...
Reading data from 2022.csv...


,year,label,total_estimate,percentage
0,2013,SEX AND AGE,NaN,0.0
1,2013,Total population,119453.0,100.0
2,2013,Male,57830.0,48.4
3,2013,Female,61623.0,51.6
4,2013,Under 5 years,6284.0,5.3
5,2013,5 to 9 years,5472.0,4.6
6,2013,10 to 14 years,5274.0,4.4
7,2013,15 to 19 years,15655.0,13.1
8,2013,20 to 24 years,21160.0,17.7
9,2013,25 to 34 years,20168.0,16.9


### Checking for data based on particular year

In [8]:
# Sample data check for specific year
year_sample = combined_df[combined_df['year'] == years[8]]
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print(year_sample)

     year                                              label  total_estimate  \
724  2022                                        SEX AND AGE             NaN   
725  2022                                   Total population        145022.0   
726  2022                                               Male         66982.0   
727  2022                                             Female         78040.0   
728  2022                  Sex ratio (males per 100 females)          4133.0   
729  2022                                      Under 5 years          6231.0   
730  2022                                       5 to 9 years          5935.0   
731  2022                                     10 to 14 years          6604.0   
732  2022                                     15 to 19 years         15784.0   
733  2022                                     20 to 24 years         20471.0   
734  2022                                     25 to 34 years         28505.0   
735  2022                               